In [1]:
import torch
import random 
import numpy as np 
from tqdm import tqdm 
from scipy.spatial.distance import cdist, cosine
from scipy.optimize import linear_sum_assignment
from utils.corrupt_graph import remove_edge, remove_node, add_edge, add_node
from utils.query_machine import get_candidates
from python_emb import *
import torch.nn.functional as F
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr, spearmanr
from collections import defaultdict
import time
from subprocess import Popen, PIPE
import pickle

# GGSX

In [2]:
def ggsx_feat(subgraph,k=6):
    counter_path_label = {}
    def pathSearch(graph: nx.Graph , node, n, path):
        ori_path = path[:]
        path.append(node)
        path_label = tuple(graph.nodes[i]['label'] for i in path)
        counter_path_label[path_label] = counter_path_label.get(path_label,0) +1
        if len(path) < n:
            for neigh in graph.neighbors(node):
                if neigh in path:
                    continue
                pathSearch(graph, neigh, n, path)
        del path[-1]
        assert path == ori_path

    path = []
    for node in subgraph.nodes():
        pathSearch(subgraph,node, k,path)
    return counter_path_label

# MCS

In [3]:
def select_label_class(future):
    labels = list(future.keys())
    values = [max(len(future[i][0]), len(future[i][1])) for i in labels]
    selected_label = labels[np.argmin(np.array(values))]
    return selected_label, future[selected_label]

def select_vertex(setG, g):
    listG = list(setG)
    values = [g.degree(i) for i in setG]
    return listG[np.argmax(np.array(values))]

def mcs(g,h):
    labelset_g = defaultdict(set)
    labelset_h = defaultdict(set)
    for node in g.nodes:
        labelset_g[g.nodes[node]['label']].add(node)
    for node in h.nodes:
        labelset_h[h.nodes[node]['label']].add(node)

    future = {}
    for label in set(labelset_g.keys()).intersection(labelset_h.keys()):
        future[str(label)+'-'] = [labelset_g[label], labelset_h[label]]
    M, incumbent = {}, {}

    def search(future):
        nonlocal incumbent, M, g,h
        if len(M) > len(incumbent):
            incumbent = M.copy()
        bound = len(M) + sum([ min(len(value[0]), len(value[1])) for value in future.values()])
        if bound <= len(incumbent):
            return 
        if len(future)==0:
            return
        selected_label, (setG, setH) = select_label_class(future)
        v = select_vertex(setG, g)
        for w in setH:
            future_ = {}
            for cur_label, (setG_, setH_) in future.items():
                setG__ = (setG_.intersection([node for node in g.neighbors(v)])).difference([v])
                setH__ = (setH_.intersection([node for node in h.neighbors(w)])).difference([w])
                if len(setG__) != 0 and len(setH__) != 0:
                    future_[cur_label+'1'] = [setG__, setH__]

                setG__ = (setG_.intersection([node for node in g.nodes if node not in g.neighbors(v)] )).difference([v])
                setH__ = (setH_.intersection([node for node in h.nodes if node not in h.neighbors(w)] )).difference([w])
                if len(setG__) != 0 and len(setH__) != 0:
                    future_[cur_label+'0'] = [setG__, setH__]
            M[v]=w
            search(future_)
            del M[v]
        setG_ = setG.difference([v])
        del future[selected_label]
        if len(setG_) > 0:
            future[selected_label] = [setG_, setH]
        search(future)
    
    search(future)
    return incumbent

# CTindex

In [4]:

def create_dimas_graph_format(graph, id_map, embedding):
    inverted_id_map = {idx:node for node,idx in id_map.items()}
    lines = ['p edge {} {} {}\n'.format(len(graph.nodes), len(graph.edges), embedding.shape[-1])]
    lines.append(' '.join(map(str, embedding)) + '\n')
    for n in range(len(inverted_id_map)):
        label = graph.nodes[inverted_id_map[n]]['label']
        lines.append('n {} {}\n'.format(n, label))
    for u, v in graph.edges():
        lines.append('e {} {}\n'.format(id_map[u], id_map[v]))
    return lines

# Graph Embedding

In [5]:
for dataset in ['yeast', 'cora', 'citeseer', 'pubmed', 'wordnet'][1:]:
    pickle_dir = '{}_pickle'.format(dataset)
    if not os.path.isdir(pickle_dir):
        os.mkdir(pickle_dir)
        
    ori_graph_data = load_data('./dataspace/graph/{}/{}'.format(dataset, dataset), supervised=False, max_degree=5, multiclass=False, use_random_walks=False)

    ori_emb_model = SupervisedGraphSage(ori_graph_data.raw_feats.shape[1], 128, ori_graph_data.num_class)
    ori_emb_model = ori_emb_model.cuda()
    ori_emb_model.load_state_dict(torch.load('model_pt/{}_sup_20.pt'.format(dataset)))
    ori_emb_model.set_params(ori_graph_data.full_adj, ori_graph_data.deg, ori_graph_data.feats)
    ori_emb_model.eval()

    ori_graph_emb = F.normalize(ori_emb_model.aggregator(list(range(ori_graph_data.raw_feats.shape[0]))), dim = 1)
    ori_graph_emb = ori_graph_emb.detach().cpu().numpy()


    for i, node in enumerate(ori_graph_data.G.nodes):
        ori_graph_data.G.nodes[node]['label'] = ori_graph_data.multi2single_label[tuple(ori_graph_data.G.nodes[node]['label'])]

    query_machine = GraphQuery(ori_emb_model, 
            ori_graph_emb,
            ori_graph_data.G,
            ori_graph_data.id_map,
            ori_graph_data.feats,
            ori_graph_data.raw_feats,
            ori_graph_data.full_adj,
            ori_graph_data.deg)

    def get_embedding_subgraph(subgraph):
        sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(subgraph)
        embedding_subgraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
        embedding_subgraph = embedding_subgraph.detach().cpu().numpy()

        return embedding_subgraph, sub_id_map


    mcs2emb = defaultdict(list)
    GRAPH_SIZE = 15
    graph_embeddings = []
    ggsx_feats = []
    ct_feats = []
    graphs = []
    nodes = list(range(len(query_machine.ori_graph)))
    random.shuffle(nodes)
    graph_id = 0
    for core_node in tqdm(nodes[:1000],desc='create graph'):
        biggraph = query_machine.create_subgraph_from_core(core_node, GRAPH_SIZE)
    #     if len(biggraph) != GRAPH_SIZE:
    #         continue
        sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(biggraph)

        embedding_biggraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
        embedding_biggraph = embedding_biggraph.detach().cpu().numpy().mean(0)
        with open('temp.graph', 'w') as f:
            for line in create_dimas_graph_format(biggraph, sub_id_map, embedding_biggraph):
                f.write(line)
        p = Popen(['timeout', '2', '../index/ctindex/emb', 'temp.graph', '.', '.', '-c'], stdin=PIPE, stdout=PIPE, stderr=PIPE)
        output, err = p.communicate()
        if len(err) > 0 or len(output) ==0 :
            continue
        ct_feat = np.zeros(128)
        ct_feat[list(set(map(int, output.strip().split())))] = 1
        ct_feats.append(ct_feat) 

        graph_embeddings.append(embedding_biggraph)
        graphs.append(biggraph)
        ggsx_f = ggsx_feat(biggraph)
        ggsx_feats.append(ggsx_f)
        with open(os.path.join(pickle_dir, str(graph_id)+'.pkl'), 'wb') as f:
            pickle.dump({'graph':biggraph, 
                         'ctindex': ct_feat,
                        'ggsx': ggsx_f,
                        'emb': embedding_biggraph}, f)
            
    graph_embeddings = np.stack(graph_embeddings)
    ct_feats = np.stack(ct_feats)


Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/yeast/yeast-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/yeast/yeast-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/yeast/yeast-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 12519
Preprocessing finished, graph info:
Name: yeast
Type: Graph
Number of nodes: 3101
Number of edges: 12519
Average degree:   8.0742


create graph: 100%|██████████| 1000/1000 [02:44<00:00,  6.10it/s]


In [109]:
n= len(graphs)
edit_distances, emb_distances, ggsx_distances = [], [], []
for _ in tqdm(range(5000)):
    i, j = random.randint(0, n-1),random.randint(0, n-1)
    graph1 = graphs[i]
    graph2 = graphs[j]
    stime=time.time()
    edit_distance = len(mcs(graph1, graph2))
    edit_distances.append(edit_distance)

    emb_distances.append( ((graph_embeddings[i] - graph_embeddings[j])**2).sum() )

    ggsx_f1 = ggsx_feats[i]
    ggsx_f2 = ggsx_feats[j]
    ggsx_distance = 0
    for k in set(ggsx_f1).union(ggsx_f2):
        ggsx_distance += abs(ggsx_f1.get(k,0) - ggsx_f2.get(k,0))
    ggsx_distances.append(ggsx_distance)










  0%|          | 0/5000 [00:00<?, ?it/s]








  0%|          | 2/5000 [00:00<04:45, 17.51it/s]








  0%|          | 9/5000 [00:00<03:42, 22.41it/s]








  0%|          | 15/5000 [00:00<03:13, 25.82it/s]








  0%|          | 18/5000 [00:00<03:07, 26.53it/s]








  0%|          | 21/5000 [00:00<06:25, 12.91it/s]








  0%|          | 25/5000 [00:01<05:17, 15.68it/s]








  1%|          | 39/5000 [00:01<04:00, 20.67it/s]








  1%|          | 54/5000 [00:01<03:01, 27.21it/s]








  1%|          | 61/5000 [00:02<06:05, 13.51it/s]








  1%|▏         | 72/5000 [00:03<05:29, 14.96it/s]








  2%|▏         | 80/5000 [00:03<05:57, 13.75it/s]








  2%|▏         | 90/5000 [00:03<04:26, 18.46it/s]








  2%|▏         | 100/5000 [00:04<03:59, 20.44it/s]








  2%|▏         | 105/5000 [00:04<03:21, 24.30it/s]








  2%|▏         | 113/5000 [00:04<03:03, 26.68it/s]








  2%|▏         | 117/5000 [00:04<03:20, 24.31it/s]








  2%|▏      

 45%|████▌     | 2254/5000 [01:06<03:21, 13.63it/s]








 45%|████▌     | 2264/5000 [01:07<03:30, 12.98it/s]








 45%|████▌     | 2271/5000 [01:07<03:02, 14.97it/s]








 46%|████▌     | 2283/5000 [01:08<02:33, 17.75it/s]








 46%|████▌     | 2295/5000 [01:08<01:57, 22.96it/s]








 46%|████▌     | 2307/5000 [01:08<01:29, 29.95it/s]








 47%|████▋     | 2327/5000 [01:08<01:06, 39.96it/s]








 47%|████▋     | 2337/5000 [01:08<01:01, 43.03it/s]








 47%|████▋     | 2346/5000 [01:09<01:32, 28.63it/s]








 47%|████▋     | 2353/5000 [01:09<01:18, 33.60it/s]








 47%|████▋     | 2360/5000 [01:10<01:45, 24.95it/s]








 47%|████▋     | 2367/5000 [01:10<01:25, 30.89it/s]








 48%|████▊     | 2377/5000 [01:10<01:07, 38.85it/s]








 48%|████▊     | 2402/5000 [01:10<00:50, 51.35it/s]








 48%|████▊     | 2413/5000 [01:10<00:46, 55.46it/s]








 49%|████▊     | 2429/5000 [01:10<00:37, 68.97it/s]








 49%|████▉     | 2441/5000 [01:10<00:45,

 89%|████████▉ | 4458/5000 [02:19<00:11, 47.41it/s]








 90%|████████▉ | 4478/5000 [02:19<00:08, 60.44it/s]








 90%|████████▉ | 4489/5000 [02:19<00:08, 60.80it/s]








 90%|████████▉ | 4499/5000 [02:19<00:07, 63.65it/s]








 90%|█████████ | 4508/5000 [02:19<00:08, 58.58it/s]








 90%|█████████ | 4522/5000 [02:19<00:08, 58.39it/s]








 91%|█████████ | 4529/5000 [02:20<00:10, 46.65it/s]








 91%|█████████ | 4543/5000 [02:20<00:08, 56.41it/s]








 91%|█████████ | 4551/5000 [02:20<00:11, 39.34it/s]








 92%|█████████▏| 4577/5000 [02:20<00:08, 51.00it/s]








 92%|█████████▏| 4590/5000 [02:21<00:09, 42.30it/s]








 92%|█████████▏| 4598/5000 [02:22<00:18, 22.07it/s]








 92%|█████████▏| 4607/5000 [02:22<00:15, 25.83it/s]








 92%|█████████▏| 4613/5000 [02:22<00:13, 28.87it/s]








 92%|█████████▏| 4618/5000 [02:22<00:11, 32.89it/s]








 93%|█████████▎| 4626/5000 [02:22<00:09, 39.65it/s]








 93%|█████████▎| 4649/5000 [02:22<00:06,

In [112]:
(spearmanr(edit_distances, emb_distances).correlation, spearmanr(edit_distances, ggsx_distances).correlation), \
(pearsonr(edit_distances, emb_distances)[0], pearsonr(edit_distances, ggsx_distances)[0])

((-0.8593299108470913, -0.13903257788467502),
 (-0.809677880513889, -0.08406750017742391))

In [6]:
n= len(graphs)
edit_matrix = np.zeros((n,n))
emb_matrix = np.zeros((n,n))
ggsx_matrix = np.zeros((n,n))
ct_matrix = np.zeros((n,n))

for i, j in tqdm([(i,j) for i in range(n) for j in range(i+1,n)],desc='cal dis'):
    graph1 = graphs[i]
    graph2 = graphs[j]
    stime=time.time()
    edit_distance = len(mcs(graph1, graph2))
    edit_matrix[i][j] = edit_distance
    edit_matrix[j][i] = edit_distance
    
    emb_distance = ((graph_embeddings[i] - graph_embeddings[j])**2).sum()
    emb_matrix[i][j] = emb_distance
    emb_matrix[j][i] = emb_distance
    
    ct_distance = ((ct_feats[i] - ct_feats[j])**2).sum()
    ct_matrix[i][j] = ct_distance
    ct_matrix[j][i] = ct_distance
    
    ggsx_f1 = ggsx_feats[i]
    ggsx_f2 = ggsx_feats[j]
    ggsx_distance = 0
    for k in set(ggsx_f1).union(ggsx_f2):
        ggsx_distance += abs(ggsx_f1.get(k,0) - ggsx_f2.get(k,0))
    ggsx_matrix[i][j] = ggsx_distance
    ggsx_matrix[j][i] = ggsx_distance

cal dis:   2%|▏         | 7898/476776 [02:42<1:57:23, 66.57it/s] 

KeyboardInterrupt: 

In [107]:
a,b = 0, 0
for i in range(n):
    a += int(spearmanr(edit_matrix[i], emb_matrix[i]).correlation < spearmanr(edit_matrix[i], ggsx_matrix[i]).correlation)
    b += int(pearsonr(edit_matrix[i], emb_matrix[i])[0] < pearsonr(edit_matrix[i], ggsx_matrix[i])[0])
a,b

(100, 100)

In [124]:
n= len(graphs)
edit_distances, emb_distances, ct_distances = [], [], []
for _ in tqdm(range(5000)):
    i, j = random.randint(0, n-1),random.randint(0, n-1)
    graph1 = graphs[i]
    graph2 = graphs[j]
    stime=time.time()
    edit_distance = len(mcs(graph1, graph2))
    edit_distances.append(edit_distance)

    emb_distances.append( ((graph_embeddings[i] - graph_embeddings[j])**2).sum() )
    ct_distances.append(((ct_feats[i] - ct_feats[j])**2).sum())











  0%|          | 0/5000 [00:00<?, ?it/s]









  0%|          | 2/5000 [00:00<04:53, 17.01it/s]









  0%|          | 5/5000 [00:00<05:48, 14.33it/s]









  0%|          | 7/5000 [00:00<08:00, 10.40it/s]









  0%|          | 10/5000 [00:00<07:09, 11.63it/s]









  1%|          | 31/5000 [00:01<05:35, 14.80it/s]









  1%|          | 34/5000 [00:01<04:57, 16.72it/s]









  1%|          | 47/5000 [00:05<10:13,  8.08it/s]









  1%|          | 53/5000 [00:05<10:06,  8.16it/s]









  1%|          | 59/5000 [00:06<11:51,  6.94it/s]









  2%|▏         | 77/5000 [00:07<08:33,  9.58it/s]









  2%|▏         | 81/5000 [00:07<09:13,  8.89it/s]









  2%|▏         | 84/5000 [00:07<07:29, 10.95it/s]









  2%|▏         | 87/5000 [00:08<07:12, 11.36it/s]









  2%|▏         | 93/5000 [00:08<05:38, 14.51it/s]









  2%|▏         | 96/5000 [00:09<13:38,  5.99it/s]









  2%|▏         | 100/5000 [00:09<11:12,  7.29it/s]









 37%|███▋      | 1855/5000 [02:04<04:59, 10.49it/s]









 37%|███▋      | 1857/5000 [02:04<06:10,  8.48it/s]









 37%|███▋      | 1873/5000 [02:04<04:28, 11.66it/s]









 38%|███▊      | 1889/5000 [02:05<03:30, 14.81it/s]









 38%|███▊      | 1893/5000 [02:05<03:34, 14.48it/s]









 38%|███▊      | 1899/5000 [02:09<12:28,  4.14it/s]









 38%|███▊      | 1902/5000 [02:10<12:11,  4.24it/s]









 38%|███▊      | 1909/5000 [02:10<09:39,  5.33it/s]









 38%|███▊      | 1914/5000 [02:10<07:29,  6.87it/s]









 38%|███▊      | 1925/5000 [02:10<05:27,  9.38it/s]









 39%|███▊      | 1932/5000 [02:11<04:23, 11.66it/s]









 39%|███▊      | 1935/5000 [02:11<05:05, 10.04it/s]









 39%|███▉      | 1942/5000 [02:12<04:34, 11.13it/s]









 39%|███▉      | 1966/5000 [02:12<03:14, 15.57it/s]









 40%|███▉      | 1975/5000 [02:12<02:59, 16.90it/s]









 40%|███▉      | 1982/5000 [02:12<02:36, 19.30it/s]









 40%|███▉      | 1992/50

 74%|███████▍  | 3698/5000 [04:02<03:22,  6.43it/s]









 74%|███████▍  | 3703/5000 [04:02<02:38,  8.19it/s]









 74%|███████▍  | 3712/5000 [04:03<02:23,  8.96it/s]









 74%|███████▍  | 3722/5000 [04:04<01:55, 11.07it/s]









 75%|███████▍  | 3733/5000 [04:04<01:25, 14.82it/s]









 75%|███████▍  | 3737/5000 [04:07<06:27,  3.26it/s]









 75%|███████▌  | 3753/5000 [04:08<04:54,  4.23it/s]









 75%|███████▌  | 3756/5000 [04:10<06:37,  3.13it/s]









 75%|███████▌  | 3758/5000 [04:11<06:26,  3.21it/s]









 75%|███████▌  | 3760/5000 [04:11<05:30,  3.75it/s]









 75%|███████▌  | 3773/5000 [04:12<04:13,  4.85it/s]









 76%|███████▌  | 3775/5000 [04:12<05:06,  3.99it/s]









 76%|███████▌  | 3782/5000 [04:13<03:43,  5.45it/s]









 76%|███████▌  | 3790/5000 [04:13<02:39,  7.57it/s]









 76%|███████▋  | 3813/5000 [04:13<01:51, 10.65it/s]









 77%|███████▋  | 3826/5000 [04:14<01:46, 11.02it/s]









 77%|███████▋  | 3836/50

In [125]:
(spearmanr(edit_distances, emb_distances).correlation, spearmanr(edit_distances, ct_distances).correlation), \
(pearsonr(edit_distances, emb_distances)[0], pearsonr(edit_distances, ct_distances)[0])

((-0.8429380198615083, -0.07337973048311028),
 (-0.8376022122139312, -0.2737486977372339))